In [20]:
import re
from urllib.request import urlopen  # b_soup_1.py
from bs4 import BeautifulSoup
import pandas as pd
from urllib.error import URLError
import pandas as pd
import requests
import socket
from urllib.error import URLError, HTTPError
from urllib.request import Request, urlopen
import os 
import pickle
from newspaper import Article
import numpy as np
from flair.models import SequenceTagger
from flair.data import Sentence
import spacy
nlp = spacy.load('en_core_web_sm')
tagger = SequenceTagger.load('ner-ontonotes-large')

2022-06-09 23:17:16,318 loading file /Users/yitingzhou/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1
2022-06-09 23:17:45,258 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [2]:
## readin all dataset
def read_all_data(name):
    all_Data = pd.read_csv(name).drop_duplicates(subset=['source_link'])
    all_Data.index.name = 'id'
    all_Data = all_Data[all_Data['source_link'].notnull()]
    return all_Data

In [3]:
def get_news_text(df):
    dates = []
    text = []
    titles = []
    idx_lst = []
    keywords = []
    summary = []
    links = []
    for i, (idx, row) in enumerate(df.iterrows()):
        article = Article(row['source_link'])
        try:
            article.download()
            article.parse()
            article.nlp()
            dates.append(article.publish_date)
            text.append(article.text)
            titles.append(article.title)
            idx_lst.append(idx)
            keywords.append(article.keywords)
            summary.append(article.summary)
            links.append(row['source_link'])
        except:
            continue
    content = {'id': idx_lst, 'links': links, 'dates': dates, 'titles': titles, 'text': text, 'keywords': keywords, 'summary': summary}  
    df = pd.DataFrame(content)
    return df

In [13]:
def filter_by_model(model, df):
    loaded_model = pickle.load(open(model, 'rb'))
    filter_text = df[df['titles'].notnull()]
    pred_res = loaded_model.predict(list(filter_text['titles']))
    filter_text['pred_res'] = pred_res
    filter_text = filter_text[filter_text['pred_res'] == 'pos']
    return filter_text

In [5]:
def get_nondate(df):
    non_dates = []
    for idx, row in df.iterrows():
        if type(df['dates']) == float:
            non_dates.append(row['id'])
    return non_dates

In [17]:
def update_date(df, non_dates):
    datepublish = []
    date_dict = {}
    for idx, i in enumerate(non_dates):
        try:
            req = Request(list(df[df['id'] == i]['links'])[0],headers={'User-Agent': 'Mozilla/5.0'})
            webpage = urlopen(req, timeout=10).read()
            if 'datepublish' in str(webpage).lower():
                text = str(webpage)[str(webpage).lower().index('datepublish'):][:300]
                sentence = Sentence(text)
                tagger.predict(sentence)
                for entity in sentence.get_spans('ner'):
                    if entity.tag == 'DATE':
                        date_dict[i] = entity.text
                        datepublish.append(i)
                        break
        except Exception:
            continue

    for idx, row in df.iterrows():
        if type(row['dates']) == float:
            if row['id'] in date_dict:
                df.loc[idx,'dates'] = date_dict[row['id']]
    return df

In [7]:
def get_trainset(df):
    GPE = []
    LOC = []
    DATE = []
    TIME = []
    INDEX = []
    SENTENCE = []
    for idx, row in df.iterrows():
        if type(row['text']) != float:
            for sent in nlp(row['text']).sents:
                sub_sent = sent.text.strip()
                if sub_sent:
                    sentence = Sentence(sub_sent)
                    tagger.predict(sentence)
                    sub_gpe = []
                    sub_date = []
                    sub_time = []
                    sub_loc = []
                    for entity in sentence.get_spans('ner'):
                        if entity.tag == 'GPE':
                            sub_gpe.append(entity.text)
                        if entity.tag == 'DATE':
                            sub_date.append(entity.text)
                        if entity.tag == 'TIME':
                            sub_time.append(entity.text)
                        if entity.tag == 'LOC':
                            sub_loc.append(entity.text)
                    if sub_loc or sub_date or sub_gpe or sub_time:
                        GPE.append('|'.join(sub_gpe))
                        LOC.append('|'.join(sub_loc))
                        DATE.append('|'.join(sub_date))
                        TIME.append('|'.join(sub_time))
                        INDEX.append(row['id'])
                        SENTENCE.append(sub_sent)
    res = pd.DataFrame({'id':INDEX, 'text': SENTENCE, 'GPE': GPE, 'LOC':LOC, 'DATE': DATE, 'TIME': TIME})
    return res

In [10]:
original_data = read_all_data('nasa_global_landslide_catalog_point.csv')
original_data

,OBJECTID,Shape,source_name,source_link,event_id,event_date,event_time,event_title,event_description,location_description,...,event_import_id,latitude,longitude,country_name,country_code,admin_division_name,gazetteer_closest_point,gazetteer_distance,submitted_date,last_edited_date
id,,,,,,,,,,,,,,,,,,,,,
0,7965080,"(73.17420000000004, 17.586100000000044)",dna,http://www.dnaindia.com/india/report-ratnagiri...,6949,2015-06-22 3:30,3:00,Dabhol village of Maharashtra's Ratnagiri dist...,At least 12 people are feared to be buried und...,Dabhol village of Maharashtra's Ratnagiri dist...,...,6949,17.5861,73.1742,India,IN,Maharashtra,Mahar Wari,18.53,2015-06-25 15:32,28:10.0
1,7965081,"(73.60750000000007, 36.25560000000007)",Pamir Times,http://pamirtimes.net/2015/10/26/earthquake-af...,7378,2015-10-26 0:00,NaN,Ghizar District,One source has put the death toll in Ghizar at...,Ghizar District,...,7378,36.2556,73.6075,Pakistan,PK,Northern Areas,Gupis,15.21,2015-12-14 15:14,28:10.0
2,7965082,"(124.10000000000002, 0.4000000000000341)",antara,http://www.antara.co.id/en/arc/2008/7/28/sulaw...,651,2008-07-17 0:00,NaN,"Bolaang Uki, Pinolosian Timur, and Pinolosian ...",Landslides and flooding throughout northern ar...,"Bolaang Uki, Pinolosian Timur, and Pinolosian ...",...,651,0.4000,124.1000,Indonesia,ID,Sulawesi Utara,Kombot,5.81,2014-04-01 0:00,2022-04-23 6:42
4,7965084,"(119.30330000000004, 28.78350000000006)",Cosumnes Connection,http://crcconnection.com/2016/10/04/typhoon-me...,9508,2016-10-04 17:00,17:00,"Sucun, Suichang, Zhejiang Province",The landslide occurred around 5 p.m. on Wednes...,"Sucun, Suichang, Zhejiang Province",...,9508,28.7835,119.3033,China,CN,Zhejiang,Miaogao,22.00,2016-10-11 20:57,2022-04-23 6:42
5,7965085,"(-122.23629999999997, 37.99580000000003)",mercurynews,http://www.mercurynews.com/breaking-news/ci_17...,3283,2011-03-24 0:00,NaN,"200 block of Carson St., Hercules, CA",03/28/2011 Authorities are closely monitoring...,"200 block of Carson St., Hercules, CA",...,3283,37.9958,-122.2363,United States,US,California,Vallejo,11.80,2014-04-01 0:00,28:10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35646,8000726,"(87.78310000000005, 27.100500000000068)",My Republica,http://www.myrepublica.com/news/2748,9185,2016-07-25 23:00,23:00,Phidim-Raanke section of the Mechi Highway at ...,Estimated location along road. The Phidim-Raan...,Phidim-Raanke section of the Mechi Highway at ...,...,9185,27.1005,87.7831,Nepal,NP,Mechi,Beteni,2.14,2016-07-26 16:00,28:10.0
35647,8000727,"(134.01570000000004, -1.2238999999999578)",Indonesia News Stand,http://indonesianewsstand.com/news/read/8358/L...,9432,2016-04-18 0:00,NaN,"Leihak, Apui, Demaisi Leihak, Sopnyai, Cangois...",A joint evacuation team has found the bodies o...,"Leihak, Apui, Demaisi Leihak, Sopnyai, Cangois...",...,9432,-1.2239,134.0157,Indonesia,ID,Papua Barat,Urongi,16.35,2016-08-05 14:49,28:10.0
35649,8000729,"(75.44810000000007, 33.126100000000065)",India TV,http://www.indiatvnews.com/news/india/landslid...,6374,2014-11-06 18:00,18:00,"Chakwa bridge, 9 kilomters short of Doda",The Doda-Batote National Highway was closed fo...,"Chakwa bridge, 9 kilomters short of Doda",...,6374,33.1261,75.4481,India,IN,Jammu and Kashmir,Doda,9.58,2014-11-07 22:36,28:10.0


In [11]:
filter_by_newspaper_date = get_news_text(original_data)
filter_by_newspaper_date


/Users/yitingzhou/opt/miniconda3/lib/python3.9/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
Building prefix dict from /Users/yitingzhou/opt/miniconda3/lib/python3.9/site-packages/jieba/dict.txt ...
Dumping model to file cache /var/folders/p2/v5_tf1952mlcvc9_pmlg59kh0000gn/T/jieba.cache
Loading model cost 1.0281739234924316 seconds.
Prefix dict has been built succesfully.
/Users/yitingzhou/opt/miniconda3/lib/python3.9/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname MDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  

,id,links,dates,titles,text,keywords,summary
0,0,http://www.dnaindia.com/india/report-ratnagiri...,None,Ratnagiri landslide: At least 12 people feared...,Reported By: | Edited By: |Source: ANI |Update...,"[maharashtra, debris, buried, feared, rescue, ...",Reported By: | Edited By: |Source: ANI |Update...
1,1,http://pamirtimes.net/2015/10/26/earthquake-af...,2015-10-26 00:00:00,"Earthquake Aftermath: 8 killed, 20 plus injure...",Gilgit/Hunza/Skardu: At least eight people wer...,"[20, injured, earthquake, killed, district, re...",Gilgit/Hunza/Skardu: At least eight people wer...
2,4,http://crcconnection.com/2016/10/04/typhoon-me...,2016-10-04 00:00:00,crcconnection.com,,[crcconnectioncom],
3,6,http://www.euronews.com/nocomment/2015/03/10/m...,2015-03-10 00:00:00,VIDEO : Landslide destroys road in Montenegro,A huge landslide in Montenegro damaged more th...,"[highway, damaged, road, montenegro, places, r...",A huge landslide in Montenegro damaged more th...
4,7,http://www.redhum.org/documento_detail/conred-...,None,América Latina y el Caribe | Topics,Office of Press Relations press@usaid.gov Yest...,"[yesterday, latina, el, caribe, américa, sidel...",Office of Press Relations press@usaid.gov Yest...
...,...,...,...,...,...,...,...
5568,35633,http://www.rappler.com/nation/80272-floods-lan...,2015-01-09 09:16:11+00:00,"Flood, landslides hit Surigao del Sur","Heavy, continuous rains cause flooding and lan...","[del, suspended, rain, classes, sur, flood, su...","Heavy, continuous rains cause flooding and lan..."
5569,35642,http://www.kcrg.com/news/local/Mudslide-Impact...,None,Local,,[local],
5570,35644,http://www.thehimalayantimes.com/fullNews.php?...,None,You are being redirected...,,[redirected],
5571,35646,http://www.myrepublica.com/news/2748,None,Mudslide blocks Mechi Highway,"PAANCHTHAR, July 26: The Phidim-Raanke section...","[mechi, highway, road, debris, vehicles, distr...","PAANCHTHAR, July 26: The Phidim-Raanke section..."


In [14]:
filter_by_model_data = filter_by_model('article_detection.model', filter_by_newspaper_date)
filter_by_model_data

,id,links,dates,titles,text,keywords,summary,pred_res
0,0,http://www.dnaindia.com/india/report-ratnagiri...,None,Ratnagiri landslide: At least 12 people feared...,Reported By: | Edited By: |Source: ANI |Update...,"[maharashtra, debris, buried, feared, rescue, ...",Reported By: | Edited By: |Source: ANI |Update...,pos
1,1,http://pamirtimes.net/2015/10/26/earthquake-af...,2015-10-26 00:00:00,"Earthquake Aftermath: 8 killed, 20 plus injure...",Gilgit/Hunza/Skardu: At least eight people wer...,"[20, injured, earthquake, killed, district, re...",Gilgit/Hunza/Skardu: At least eight people wer...,pos
3,6,http://www.euronews.com/nocomment/2015/03/10/m...,2015-03-10 00:00:00,VIDEO : Landslide destroys road in Montenegro,A huge landslide in Montenegro damaged more th...,"[highway, damaged, road, montenegro, places, r...",A huge landslide in Montenegro damaged more th...,pos
8,19,https://www.kelownanow.com/watercooler/news/ne...,None,"Rockslide, Travel Advisory, and Closed Highway...",Highways throughout B.C. continue to cause hea...,"[slide, problems, causing, highway, motorists,...",The highway nine kilometres west of Keremeos i...,pos
9,20,http://www.guardian.co.uk/edinburgh/2010/mar/3...,2010-03-31 00:00:00,Travel disruption latest: No East Coast trains,Train travel to and from the city is severly d...,"[rain, train, coast, expected, landslips, snow...",Train travel to and from the city is severly d...,pos
...,...,...,...,...,...,...,...,...
5563,35609,http://twocircles.net/2012jun22/landslide_kill...,2012-06-22 13:55:56+05:30,Landslide kills three in Assam,"Help India!\n\nBy IANS,\n\nGuwahati : A man an...","[victims, took, twocirclesthe, saidsupport, ni...","By IANS,Guwahati : A man and his two minor son...",pos
5564,35610,http://www.reporterherald.com/ci_23649025/weat...,2013-07-12 15:10:08+00:00,Mudslides close Poudre Canyon highway at Ted’s...,The Larimer County Sheriff’s Office reported a...,"[close, rain, highway, mudslides, road, flash,...",The Larimer County Sheriff’s Office reported a...,pos
5568,35633,http://www.rappler.com/nation/80272-floods-lan...,2015-01-09 09:16:11+00:00,"Flood, landslides hit Surigao del Sur","Heavy, continuous rains cause flooding and lan...","[del, suspended, rain, classes, sur, flood, su...","Heavy, continuous rains cause flooding and lan...",pos
5571,35646,http://www.myrepublica.com/news/2748,None,Mudslide blocks Mechi Highway,"PAANCHTHAR, July 26: The Phidim-Raanke section...","[mechi, highway, road, debris, vehicles, distr...","PAANCHTHAR, July 26: The Phidim-Raanke section...",pos


In [18]:
non_date_dict = get_nondate(filter_by_model_data)
final_filter_date = update_date(filter_by_model_data, non_date_dict)
final_filter_date

,id,links,dates,titles,text,keywords,summary,pred_res
0,0,http://www.dnaindia.com/india/report-ratnagiri...,None,Ratnagiri landslide: At least 12 people feared...,Reported By: | Edited By: |Source: ANI |Update...,"[maharashtra, debris, buried, feared, rescue, ...",Reported By: | Edited By: |Source: ANI |Update...,pos
1,1,http://pamirtimes.net/2015/10/26/earthquake-af...,2015-10-26 00:00:00,"Earthquake Aftermath: 8 killed, 20 plus injure...",Gilgit/Hunza/Skardu: At least eight people wer...,"[20, injured, earthquake, killed, district, re...",Gilgit/Hunza/Skardu: At least eight people wer...,pos
3,6,http://www.euronews.com/nocomment/2015/03/10/m...,2015-03-10 00:00:00,VIDEO : Landslide destroys road in Montenegro,A huge landslide in Montenegro damaged more th...,"[highway, damaged, road, montenegro, places, r...",A huge landslide in Montenegro damaged more th...,pos
8,19,https://www.kelownanow.com/watercooler/news/ne...,None,"Rockslide, Travel Advisory, and Closed Highway...",Highways throughout B.C. continue to cause hea...,"[slide, problems, causing, highway, motorists,...",The highway nine kilometres west of Keremeos i...,pos
9,20,http://www.guardian.co.uk/edinburgh/2010/mar/3...,2010-03-31 00:00:00,Travel disruption latest: No East Coast trains,Train travel to and from the city is severly d...,"[rain, train, coast, expected, landslips, snow...",Train travel to and from the city is severly d...,pos
...,...,...,...,...,...,...,...,...
5563,35609,http://twocircles.net/2012jun22/landslide_kill...,2012-06-22 13:55:56+05:30,Landslide kills three in Assam,"Help India!\n\nBy IANS,\n\nGuwahati : A man an...","[victims, took, twocirclesthe, saidsupport, ni...","By IANS,Guwahati : A man and his two minor son...",pos
5564,35610,http://www.reporterherald.com/ci_23649025/weat...,2013-07-12 15:10:08+00:00,Mudslides close Poudre Canyon highway at Ted’s...,The Larimer County Sheriff’s Office reported a...,"[close, rain, highway, mudslides, road, flash,...",The Larimer County Sheriff’s Office reported a...,pos
5568,35633,http://www.rappler.com/nation/80272-floods-lan...,2015-01-09 09:16:11+00:00,"Flood, landslides hit Surigao del Sur","Heavy, continuous rains cause flooding and lan...","[del, suspended, rain, classes, sur, flood, su...","Heavy, continuous rains cause flooding and lan...",pos
5571,35646,http://www.myrepublica.com/news/2748,None,Mudslide blocks Mechi Highway,"PAANCHTHAR, July 26: The Phidim-Raanke section...","[mechi, highway, road, debris, vehicles, distr...","PAANCHTHAR, July 26: The Phidim-Raanke section...",pos


In [23]:
training_date = get_trainset(final_filter_date)
training_date

,id,text,GPE,LOC,DATE,TIME
0,0,Reported By: | Edited By: |Source: ANI |Update...,Dabhol|Maharashtra|Ratnagiri,,"Aug 04 , 2015",11:39 AM
1,0,The landslide is said to have taken place at 3...,,,,3:30 am
2,0,Ratnagiri (Maharashtra): Landslide occurred at...,Ratnagiri|Maharashtra,,today,around 3 : 30 am
3,0,"pic.twitter.com/MOcmTxnAZX June 22, 2015\n\nSp...",Dabhol Village|Ratnagiri|Maharashtra,,"June 22 , 2015",
4,0,pic.twitter.com/a5mMtI2hwB — ANI (@ANI_news) J...,,,"June 22 , 2015",Earlier in the day
...,...,...,...,...,...,...
19631,35646,The district authorities stated that the road ...,,,today,
19632,35646,"Meanwhile, the Kanchanjunga Academy Higher Sec...",,Phidim-1,,
19633,35649,Follow us on landslide blocks doda batote high...,Jammu,,,
19634,35649,The highway was blocked after a landslide hit ...,Chakwa|Doda,,,last evening
